# Importation des données

In [1]:
import sqlite3
import pandas as pd

In [2]:
connect = sqlite3.connect("../Importation_donnees/raw-database.db")
cursor = connect.cursor()

In [3]:
req = cursor.execute(f"SELECT * FROM freeze_frames")
res = req.fetchall()
desc = req.description
freeze_frames = pd.DataFrame(res)
freeze_frames.columns = [i[0] for i in desc]

---
# Etudes des colonnes comportant des valeurs Nan

In [4]:
freeze_frames.isna().any()

frame              False
timestamp           True
period              True
event_id           False
event_x            False
event_y            False
is_matched         False
match_id_SKC       False
group               True
tackable_object     True
dtype: bool

---
# Etudes des valeurs Nan pour la colonne "timestamp"

In [5]:
# Index des frames correspondants a des valeurs Nan
index = freeze_frames.index[freeze_frames.timestamp.isna()]

In [6]:
# Index des frames correspondant au valeur Nan, ainsi que les frames précédentes et suivantes de ces dernières.
freeze_frames.loc[index.tolist() + (index - 1).tolist() + (index + 1).tolist()].sort_index()

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object
12999,27215,2025-02-05 00:44:53.500000,1.0,cc95d2bf-4f94-47dc-aa8c-82cd509905f9,-39.8125,20.06,1,1404545,away team,NaN
13000,27888,None,NaN,37b9dbf6-1cbb-42b8-b5c3-e1b3934b596e,unknown,unknown,0,1404545,away team,25539.0
13001,37834,2025-02-05 00:47:00.400000,2.0,2d818cdd-e513-405c-bbeb-dd57f8081774,-46.725,-21.505,1,1404545,home team,9451.0
16739,33105,2025-02-05 00:44:52.500000,1.0,c1ad7f3b-7ec6-45c3-9745-ae79cacee74b,34.0375,20.06,1,1483175,away team,NaN
16740,43538,None,NaN,f5f29ce5-9e11-465c-85ac-4c69a1b30379,unknown,unknown,0,1483175,None,28768.0
16741,45712,2025-02-05 00:48:37.200000,2.0,985ec082-3c00-4ab7-bb39-4e0f5979d445,-20.125,28.645,1,1483175,away team,NaN
17761,25577,2025-02-05 00:32:19.700000,1.0,b5d8596c-ae2a-4061-929b-6e9df0f95ff4,-49.6125,-10.03,1,1499186,home team,NaN
17762,33842,None,NaN,5dc500e7-876c-41bd-b353-139debc91e50,unknown,unknown,0,1499186,None,NaN
17763,44797,2025-02-05 00:47:01.700000,2.0,914419e5-7a05-4e95-8a8c-7d72eb1efbd1,-42.2625,-26.775,0,1499186,away team,NaN
20175,73572,2025-02-05 01:32:06.200000,2.0,99d2acb7-3a12-4c50-b129-9ffeca155293,-21.875,29.07,1,1547882,home team,NaN


In [7]:
# Importation des events SB correspondants aux frames Nan
params = freeze_frames.loc[index.tolist() + (index - 1).tolist() + (index + 1).tolist()].sort_index().event_id.tolist()
stat = f"SELECT * FROM events WHERE event_id IN ({', '.join('?' * len(params))})"
req = cursor.execute(stat, params)
res = req.fetchall()
desc = req.description
events = pd.DataFrame(res)
events.columns = [i[0] for i in desc]

In [8]:
events.drop(["index_event", "shot_outcome", "y_loc", "z_loc", "y_pass", "y_shot", "z_shot"], axis = 1).sort_values(by = ["match_id_SB", "minute"])

,event_id,shot_type,type,match_id_SB,period,possession,pass_cross,pass_type,minute,pass_body_part,player_id_SB,team_id_SB,x_loc,x_pass,x_shot
0,cc95d2bf-4f94-47dc-aa8c-82cd509905f9,None,Pressure,3894262,1,68,NaN,None,44,None,5507.0,137,14.5,NaN,None
1,37b9dbf6-1cbb-42b8-b5c3-e1b3934b596e,None,Half End,3894262,1,71,NaN,None,45,None,NaN,136,NaN,NaN,None
2,2d818cdd-e513-405c-bbeb-dd57f8081774,None,Pass,3894262,2,75,1.0,None,46,Left Foot,7212.0,137,113.4,104.2,None
3,c1ad7f3b-7ec6-45c3-9745-ae79cacee74b,None,Pass,3894316,1,74,1.0,None,44,Right Foot,4909.0,136,98.9,120.0,None
4,f5f29ce5-9e11-465c-85ac-4c69a1b30379,None,Half Start,3894316,2,77,NaN,None,45,None,NaN,156,NaN,NaN,None
5,985ec082-3c00-4ab7-bb39-4e0f5979d445,None,Carry,3894316,2,86,NaN,None,48,None,46009.0,136,83.0,NaN,None
6,b5d8596c-ae2a-4061-929b-6e9df0f95ff4,None,Clearance,3894335,1,66,NaN,None,32,None,22137.0,152,3.3,NaN,None
7,5dc500e7-876c-41bd-b353-139debc91e50,None,Half End,3894335,1,92,NaN,None,46,None,NaN,168,NaN,NaN,None
8,914419e5-7a05-4e95-8a8c-7d72eb1efbd1,None,Pass,3894335,2,97,1.0,None,47,Right Foot,23682.0,152,108.3,106.7,None
9,99d2acb7-3a12-4c50-b129-9ffeca155293,None,Pass,3894368,2,184,1.0,None,92,Right Foot,3363.0,144,85.0,105.8,None


Les valeurs Nan de cette colonne semblent correspondre a des évènements qui référencient une fin de période

---
# Etudes des valeurs Nan pour la colonne "period"

In [9]:
# Index des frames correspondants a des valeurs Nan
index = freeze_frames.index[freeze_frames.period.isna()]

In [10]:
# Index des frames correspondant au valeur Nan, ainsi que les frames précédentes et suivantes de ces dernières.
freeze_frames.loc[index.tolist() + (index - 1).tolist() + (index + 1).tolist()].sort_index()

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object
12999,27215,2025-02-05 00:44:53.500000,1.0,cc95d2bf-4f94-47dc-aa8c-82cd509905f9,-39.8125,20.06,1,1404545,away team,NaN
13000,27888,None,NaN,37b9dbf6-1cbb-42b8-b5c3-e1b3934b596e,unknown,unknown,0,1404545,away team,25539.0
13001,37834,2025-02-05 00:47:00.400000,2.0,2d818cdd-e513-405c-bbeb-dd57f8081774,-46.725,-21.505,1,1404545,home team,9451.0
16739,33105,2025-02-05 00:44:52.500000,1.0,c1ad7f3b-7ec6-45c3-9745-ae79cacee74b,34.0375,20.06,1,1483175,away team,NaN
16740,43538,None,NaN,f5f29ce5-9e11-465c-85ac-4c69a1b30379,unknown,unknown,0,1483175,None,28768.0
16741,45712,2025-02-05 00:48:37.200000,2.0,985ec082-3c00-4ab7-bb39-4e0f5979d445,-20.125,28.645,1,1483175,away team,NaN
17761,25577,2025-02-05 00:32:19.700000,1.0,b5d8596c-ae2a-4061-929b-6e9df0f95ff4,-49.6125,-10.03,1,1499186,home team,NaN
17762,33842,None,NaN,5dc500e7-876c-41bd-b353-139debc91e50,unknown,unknown,0,1499186,None,NaN
17763,44797,2025-02-05 00:47:01.700000,2.0,914419e5-7a05-4e95-8a8c-7d72eb1efbd1,-42.2625,-26.775,0,1499186,away team,NaN
20175,73572,2025-02-05 01:32:06.200000,2.0,99d2acb7-3a12-4c50-b129-9ffeca155293,-21.875,29.07,1,1547882,home team,NaN


In [11]:
# Importation des events SB correspondants aux frames Nan
params = freeze_frames.loc[index.tolist() + (index - 1).tolist() + (index + 1).tolist()].sort_index().event_id.tolist()
stat = f"SELECT * FROM events WHERE event_id IN ({', '.join('?' * len(params))})"
req = cursor.execute(stat, params)
res = req.fetchall()
desc = req.description
events = pd.DataFrame(res)
events.columns = [i[0] for i in desc]

In [12]:
events.drop(["index_event", "shot_outcome", "y_loc", "z_loc", "y_pass", "y_shot", "z_shot"], axis = 1).sort_values(by = ["match_id_SB", "minute"])

,event_id,shot_type,type,match_id_SB,period,possession,pass_cross,pass_type,minute,pass_body_part,player_id_SB,team_id_SB,x_loc,x_pass,x_shot
0,cc95d2bf-4f94-47dc-aa8c-82cd509905f9,None,Pressure,3894262,1,68,NaN,None,44,None,5507.0,137,14.5,NaN,None
1,37b9dbf6-1cbb-42b8-b5c3-e1b3934b596e,None,Half End,3894262,1,71,NaN,None,45,None,NaN,136,NaN,NaN,None
2,2d818cdd-e513-405c-bbeb-dd57f8081774,None,Pass,3894262,2,75,1.0,None,46,Left Foot,7212.0,137,113.4,104.2,None
3,c1ad7f3b-7ec6-45c3-9745-ae79cacee74b,None,Pass,3894316,1,74,1.0,None,44,Right Foot,4909.0,136,98.9,120.0,None
4,f5f29ce5-9e11-465c-85ac-4c69a1b30379,None,Half Start,3894316,2,77,NaN,None,45,None,NaN,156,NaN,NaN,None
5,985ec082-3c00-4ab7-bb39-4e0f5979d445,None,Carry,3894316,2,86,NaN,None,48,None,46009.0,136,83.0,NaN,None
6,b5d8596c-ae2a-4061-929b-6e9df0f95ff4,None,Clearance,3894335,1,66,NaN,None,32,None,22137.0,152,3.3,NaN,None
7,5dc500e7-876c-41bd-b353-139debc91e50,None,Half End,3894335,1,92,NaN,None,46,None,NaN,168,NaN,NaN,None
8,914419e5-7a05-4e95-8a8c-7d72eb1efbd1,None,Pass,3894335,2,97,1.0,None,47,Right Foot,23682.0,152,108.3,106.7,None
9,99d2acb7-3a12-4c50-b129-9ffeca155293,None,Pass,3894368,2,184,1.0,None,92,Right Foot,3363.0,144,85.0,105.8,None


De même que pour la colonne "timestamp", les frames Nan correspondent à des débuts/fins de périodes

---
# Etudes des valeurs Nan pour la colonne "group"

In [13]:
# Index des frames correspondants a des valeurs Nan
index = freeze_frames.index[freeze_frames.group.isna()]

In [14]:
freeze_frames.loc[index].event_id.head(10)

3      c1a7a598-9690-4a32-a733-b7b3b585df83
4      d3b91062-3398-4235-8bbc-3571d2fd2d9c
27     b94a4ca1-d637-4cfc-a6ad-4fb6f71c95c6
53     f9bde630-3159-4e49-9dc3-93873bdb37ee
96     4e6b6d44-2c5c-4585-bd86-1aef4aeacbbb
111    bae5c26c-8572-46cf-afca-b10d3693bbd5
131    c87aaee5-70a3-4cb3-8fda-b462a52109cf
225    a80fb33a-1e68-4654-a241-8771822d23b3
226    d8bf82d2-6674-4bf7-b5c8-9915d40121bf
227    d5a94c29-f3c9-4aad-a4a5-3d3720cb56d9
Name: event_id, dtype: object

In [15]:
# Index des frames correspondant au valeur Nan, ainsi que les frames précédentes et suivantes de ces dernières.
multi_frames = freeze_frames.loc[index.tolist() + (index - 1).tolist() + (index + 1).tolist()].sort_index().drop_duplicates()

In [16]:
multi_frames

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object
2,4854,2025-02-05 00:04:32.400000,1.0,3ea6ecd9-bc39-4ce5-9b47-1eda239f2705,44.1,1.615,1,1020089,away team,NaN
3,5752,2025-02-05 00:06:02.200000,1.0,c1a7a598-9690-4a32-a733-b7b3b585df83,-43.4,-27.455,1,1020089,None,NaN
4,5755,2025-02-05 00:06:02.500000,1.0,d3b91062-3398-4235-8bbc-3571d2fd2d9c,-43.75,-29.24,0,1020089,None,NaN
5,5796,2025-02-05 00:06:06.600000,1.0,1ea8cb69-9095-4a7e-bc1a-e1ca8c4984b6,-48.825,28.475,1,1020089,home team,NaN
26,27860,2025-02-05 00:42:53,1.0,c18d6b9e-4113-4157-a57d-aee79d0f2f51,46.55,10.71,1,1020089,home team,12327.0
...,...,...,...,...,...,...,...,...,...,...
20478,69055,2025-02-05 01:26:25.500000,2.0,1b48bd45-91c9-44ca-a4d2-5cc26510b359,26.95,-22.185,1,1547887,None,NaN
20479,69067,2025-02-05 01:26:26.700000,2.0,55cebbd1-a06e-4595-8d4d-db9e9f74d529,42.35,-4.165,1,1547887,home team,NaN
20538,50122,2025-02-05 00:52:46.200000,2.0,c7284660-2243-4117-8e27-f6ae465e38f3,40.1625,-30.175,0,1547888,home team,NaN
20539,50177,2025-02-05 00:52:51.700000,2.0,f5234205-3b40-4aa4-a39d-2aae0f7c99cd,48.2125,33.915,0,1547888,None,NaN


In [17]:
# Importation des events SB correspondants aux frames Nan
params = multi_frames.event_id.tolist()
stat = f"SELECT * FROM events WHERE event_id IN ({', '.join('?' * len(params))})"
req = cursor.execute(stat, params)
res = req.fetchall()
desc = req.description
events = pd.DataFrame(res)
events.columns = [i[0] for i in desc]

In [18]:
events.drop(["index_event", "shot_outcome", "y_loc", "z_loc", "y_pass", "y_shot", "z_shot"], axis = 1)[events.event_id.isin(freeze_frames.loc[index].event_id)]

,event_id,shot_type,type,match_id_SB,period,possession,pass_cross,pass_type,minute,pass_body_part,player_id_SB,team_id_SB,x_loc,x_pass,x_shot
1,4e6b6d44-2c5c-4585-bd86-1aef4aeacbbb,None,Player Off,3894037,1,68,NaN,None,33,None,316514.0,147,NaN,NaN,None
4,bae5c26c-8572-46cf-afca-b10d3693bbd5,None,Substitution,3894037,2,121,NaN,None,59,None,212489.0,156,NaN,NaN,None
7,c87aaee5-70a3-4cb3-8fda-b462a52109cf,None,Ball Receipt*,3894037,2,170,NaN,None,88,None,3712.0,147,104.0,NaN,None
9,e4ec3204-6ddb-4202-9743-6ce36a43bd2a,None,Injury Stoppage,3894038,2,82,NaN,None,46,None,4448.0,141,NaN,NaN,None
12,0dfc0964-d330-45e3-af1a-74e053e59a4e,None,Ball Receipt*,3894038,2,130,NaN,None,81,None,28238.0,141,91.6,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,9d6ff703-8ce0-4aa9-a650-da819e1d142b,None,Ball Receipt*,3894370,1,31,NaN,None,18,None,3451.0,137,114.6,NaN,None
1496,700c61dd-2608-4497-8355-fc143d2c2043,None,Pass,3894370,1,49,NaN,Goal Kick,27,Right Foot,2947.0,137,6.0,12.4,None
1499,873a5f6b-ff24-43db-a983-a88164800daf,None,Pressure,3894371,1,36,NaN,None,17,None,29649.0,156,105.3,NaN,None
1502,1b48bd45-91c9-44ca-a4d2-5cc26510b359,None,Pass,3894373,2,174,1.0,Free Kick,86,Right Foot,4413.0,164,90.8,108.5,None


Les valeurs Nan de cette colonne correspondent à des events ou le ballon est en dehors du terrain

---
# Etudes des valeurs Nan pour la colonne "tackable_object"

In [19]:
# Index des frames correspondants a des valeurs Nan
index = freeze_frames.index[freeze_frames.tackable_object.isna()]

In [20]:
freeze_frames.loc[index].event_id

0        54cbad0c-1f05-4708-9b7f-2cadeae0636b
2        3ea6ecd9-bc39-4ce5-9b47-1eda239f2705
3        c1a7a598-9690-4a32-a733-b7b3b585df83
4        d3b91062-3398-4235-8bbc-3571d2fd2d9c
5        1ea8cb69-9095-4a7e-bc1a-e1ca8c4984b6
                         ...                 
20571    4667ef16-2566-41b8-98d9-4606aabf5d05
20572    ece56aa9-a9c7-48b5-8dbb-aa043cdec502
20573    4b410e07-5965-4dc0-b4e0-4664b770413b
20574    bf588ed7-391f-48d6-ab20-cd38b3e571da
20575    5954f7ff-0805-41e3-af67-b277cb4343ce
Name: event_id, Length: 15473, dtype: object

In [21]:
# Index des frames correspondant au valeur Nan.
multi_frames = freeze_frames.loc[index.tolist()].sort_index()

Les valeurs Nan de cette colonne correspondent à des events ou aucune joueur n'a le ballon